# Previsão de Churn na Telecom X

Este notebook contém a análise completa para prever a evasão (churn) de clientes da Telecom X.

## Objetivos:
- Preparar os dados para modelagem
- Realizar análise exploratória
- Treinar modelos de classificação
- Avaliar o desempenho dos modelos
- Interpretar os resultados

## 1. Importação das Bibliotecas

In [211]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Configurações de visualização
plt.style.use('default')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Carregamento e Pré-processamento dos Dados

In [212]:
import os

# Carregar os dados
if os.path.exists(file_path):
	df = pd.read_json(file_path)
	print("Primeiras 5 linhas do DataFrame original:")
	print(df.head())
	print(f"\nShape dos dados: {df.shape}")
	print(f"\nColunas: {df.columns.tolist()}")
else:
	print(f"Arquivo não encontrado: {file_path}")
	df = None

Arquivo não encontrado: ../data/TelecomX_Data.json


In [213]:
# Função para achatar colunas JSON aninhadas
def flatten_json_col(df, col_name):
    col_df = pd.json_normalize(df[col_name])
    df = df.drop(col_name, axis=1)
    df = pd.concat([df, col_df], axis=1)
    return df

if df is not None:
    # Achatar as colunas aninhadas
    df = flatten_json_col(df, 'customer')
    df = flatten_json_col(df, 'phone')
    df = flatten_json_col(df, 'internet')

    # Achatar a coluna 'account' primeiro para acessar 'Charges'
    df_account = pd.json_normalize(df['account'])
    df = df.drop('account', axis=1)
    df = pd.concat([df, df_account], axis=1)

    # Renomear as colunas 'Charges.Monthly' e 'Charges.Total'
    df = df.rename(columns={'Charges.Monthly': 'MonthlyCharges', 'Charges.Total': 'TotalCharges'})

    print("Colunas após o pré-processamento:")
    print(df.columns.tolist())
else:
    print("DataFrame 'df' está vazio. Verifique se o arquivo foi carregado corretamente.")

DataFrame 'df' está vazio. Verifique se o arquivo foi carregado corretamente.


In [214]:
if df is not None:
	# Remover a coluna de ID do cliente
	df = df.drop('customerID', axis=1)

	# Tratar valores ausentes na coluna 'Churn'
	df['Churn'] = df['Churn'].replace('', 'No')

	# Converter a coluna 'TotalCharges' para numérica
	df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
	df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

	print("Verificando valores nulos após limpeza:")
	print(df.isnull().sum())

	print("\nDistribuição da variável target (Churn):")
	print(df['Churn'].value_counts())
else:
	print("DataFrame 'df' está vazio. Verifique se o arquivo foi carregado corretamente.")

DataFrame 'df' está vazio. Verifique se o arquivo foi carregado corretamente.


In [215]:
# Codificar variáveis categóricas usando one-hot encoding
if df is not None:
	categorical_cols = df.select_dtypes(include=['object']).columns
	df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

	print(f"Shape após encoding: {df_encoded.shape}")
	print("\nPrimeiras 5 linhas do DataFrame processado:")
	print(df_encoded.head())
else:
	print("DataFrame 'df' está vazio. Verifique se o arquivo foi carregado corretamente.")

DataFrame 'df' está vazio. Verifique se o arquivo foi carregado corretamente.


## 3. Análise Exploratória dos Dados

In [216]:
# Verificar se df_encoded está definido
if 'df_encoded' in globals():
	# Verificar a proporção de churn
	churn_counts = df_encoded['Churn_Yes'].value_counts()
	print("Proporção de Churn:")
	print(f"Não churn (0): {churn_counts[0]} ({churn_counts[0]/len(df_encoded)*100:.2f}%)")
	print(f"Churn (1): {churn_counts[1]} ({churn_counts[1]/len(df_encoded)*100:.2f}%)")

	# Visualizar a distribuição de churn
	plt.figure(figsize=(8, 6))
	churn_counts.plot(kind='bar')
	plt.title('Distribuição de Churn')
	plt.xlabel('Churn (0=Não, 1=Sim)')
	plt.ylabel('Quantidade de Clientes')
	plt.xticks(rotation=0)
	plt.show()
else:
	print("Erro: df_encoded não está definido. Verifique se o arquivo foi carregado corretamente e se o pré-processamento foi realizado.")

Erro: df_encoded não está definido. Verifique se o arquivo foi carregado corretamente e se o pré-processamento foi realizado.


In [217]:
# Certifique-se de que df_encoded está definido
if 'df_encoded' in globals():
	# Calcular a matriz de correlação
	correlation_matrix = df_encoded.corr()
else:
	print("Erro: df_encoded não está definido. Verifique se o arquivo foi carregado corretamente e se o pré-processamento foi realizado.")
	correlation_matrix = None

# Visualizar a matriz de correlação
if correlation_matrix is not None:
	plt.figure(figsize=(20, 16))
	sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0)
	plt.title('Matriz de Correlação')
	plt.xticks(rotation=45, ha='right')
	plt.yticks(rotation=0)
	plt.tight_layout()
	plt.show()
else:
	print("Erro: Não foi possível calcular a matriz de correlação.")

Erro: df_encoded não está definido. Verifique se o arquivo foi carregado corretamente e se o pré-processamento foi realizado.
Erro: Não foi possível calcular a matriz de correlação.


In [218]:
# Correlação com Churn
if correlation_matrix is not None:
	churn_corr = correlation_matrix['Churn_Yes'].sort_values(ascending=False)
	churn_corr = churn_corr[churn_corr.index != 'Churn_Yes']  # Remove a própria variável

	print("Top 15 correlações positivas com Churn:")
	print(churn_corr.head(15))
	print("\nTop 15 correlações negativas com Churn:")
	print(churn_corr.tail(15))

	# Visualizar as correlações mais importantes
	top_corr = churn_corr.head(10)
	bottom_corr = churn_corr.tail(10)
	combined_corr = pd.concat([top_corr, bottom_corr])

	plt.figure(figsize=(12, 8))
	plt.barh(range(len(combined_corr)), combined_corr.values)
	plt.yticks(range(len(combined_corr)), combined_corr.index)
	plt.xlabel('Correlação com Churn')
	plt.title('Top 10 Correlações Positivas e Negativas com Churn')
	plt.grid(axis='x', alpha=0.3)
	plt.tight_layout()
	plt.show()
else:
	print("Erro: correlation_matrix não está definido. Execute a célula de cálculo da matriz de correlação antes de rodar esta análise.")

Erro: correlation_matrix não está definido. Execute a célula de cálculo da matriz de correlação antes de rodar esta análise.


In [219]:
# Análise de variáveis específicas
if 'df_encoded' in globals():
	fig, axes = plt.subplots(2, 2, figsize=(15, 12))

	# Distribuição de MonthlyCharges por Churn
	churn_0 = df_encoded[df_encoded['Churn_Yes'] == 0]['MonthlyCharges']
	churn_1 = df_encoded[df_encoded['Churn_Yes'] == 1]['MonthlyCharges']
	axes[0, 0].hist(churn_0, alpha=0.7, label='Não Churn', bins=30, density=True)
	axes[0, 0].hist(churn_1, alpha=0.7, label='Churn', bins=30, density=True)
	axes[0, 0].set_xlabel('MonthlyCharges')
	axes[0, 0].set_ylabel('Densidade')
	axes[0, 0].set_title('Distribuição de MonthlyCharges por Churn')
	axes[0, 0].legend()

	# Distribuição de tenure por Churn
	churn_0_tenure = df_encoded[df_encoded['Churn_Yes'] == 0]['tenure']
	churn_1_tenure = df_encoded[df_encoded['Churn_Yes'] == 1]['tenure']
	axes[0, 1].hist(churn_0_tenure, alpha=0.7, label='Não Churn', bins=30, density=True)
	axes[0, 1].hist(churn_1_tenure, alpha=0.7, label='Churn', bins=30, density=True)
	axes[0, 1].set_xlabel('Tenure (meses)')
	axes[0, 1].set_ylabel('Densidade')
	axes[0, 1].set_title('Distribuição de Tenure por Churn')
	axes[0, 1].legend()

	# Distribuição de TotalCharges por Churn
	churn_0_total = df_encoded[df_encoded['Churn_Yes'] == 0]['TotalCharges']
	churn_1_total = df_encoded[df_encoded['Churn_Yes'] == 1]['TotalCharges']
	axes[1, 0].hist(churn_0_total, alpha=0.7, label='Não Churn', bins=30, density=True)
	axes[1, 0].hist(churn_1_total, alpha=0.7, label='Churn', bins=30, density=True)
	axes[1, 0].set_xlabel('TotalCharges')
	axes[1, 0].set_ylabel('Densidade')
	axes[1, 0].set_title('Distribuição de TotalCharges por Churn')
	axes[1, 0].legend()

	# Boxplot de MonthlyCharges por Churn
	df_encoded.boxplot(column='MonthlyCharges', by='Churn_Yes', ax=axes[1, 1])
	axes[1, 1].set_title('Boxplot de MonthlyCharges por Churn')
	axes[1, 1].set_xlabel('Churn (0=Não, 1=Sim)')

	plt.tight_layout()
	plt.show()
else:
	print("Erro: df_encoded não está definido. Verifique se o arquivo foi carregado corretamente e se o pré-processamento foi realizado.")

Erro: df_encoded não está definido. Verifique se o arquivo foi carregado corretamente e se o pré-processamento foi realizado.


## 4. Desenvolvimento dos Modelos de Machine Learning

In [220]:
# Separar features (X) e target (y) somente se df_encoded estiver definido
if 'df_encoded' in globals():
	X = df_encoded.drop("Churn_Yes", axis=1)
	y = df_encoded["Churn_Yes"]

	print(f"Shape das features: {X.shape}")
	print(f"Shape do target: {y.shape}")

	# Dividir os dados em treino e teste
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

	print(f"\nShape do conjunto de treino: {X_train.shape}")
	print(f"Shape do conjunto de teste: {X_test.shape}")
else:
	print("Erro: df_encoded não está definido. Execute as células de pré-processamento antes de rodar esta célula.")

Erro: df_encoded não está definido. Execute as células de pré-processamento antes de rodar esta célula.


In [221]:
# Normalização dos dados para modelos sensíveis à escala
if 'X_train' in globals() and 'X_test' in globals():
	scaler = StandardScaler()
	X_train_scaled = scaler.fit_transform(X_train)
	X_test_scaled = scaler.transform(X_test)

	print("Dados normalizados com sucesso!")
	print(f"Média das features normalizadas (treino): {X_train_scaled.mean(axis=0)[:5]}")
	print(f"Desvio padrão das features normalizadas (treino): {X_train_scaled.std(axis=0)[:5]}")
else:
	print("Erro: X_train e/ou X_test não estão definidos. Execute a célula de separação dos dados antes de rodar esta célula.")

Erro: X_train e/ou X_test não estão definidos. Execute a célula de separação dos dados antes de rodar esta célula.


In [222]:
# Modelo 1: Regressão Logística (sensível à escala)
if 'X_train_scaled' in globals() and 'X_test_scaled' in globals():
	print("Treinando Regressão Logística...")
	log_reg_model = LogisticRegression(random_state=42, solver='liblinear')
	log_reg_model.fit(X_train_scaled, y_train)
	y_pred_log_reg = log_reg_model.predict(X_test_scaled)
	print("Regressão Logística treinada com sucesso!")
else:
	print("Erro: X_train_scaled e/ou X_test_scaled não estão definidos. Execute a célula de normalização antes de rodar esta célula.")

Erro: X_train_scaled e/ou X_test_scaled não estão definidos. Execute a célula de normalização antes de rodar esta célula.


In [223]:
# Modelo 2: Random Forest (não sensível à escala)
if 'X_train' in globals() and 'y_train' in globals() and 'X_test' in globals():
	print("Treinando Random Forest...")
	rf_model = RandomForestClassifier(random_state=42)
	rf_model.fit(X_train, y_train)
	y_pred_rf = rf_model.predict(X_test)
	print("Random Forest treinado com sucesso!")
else:
	print("Erro: X_train, y_train e/ou X_test não estão definidos. Execute a célula de separação dos dados antes de rodar esta célula.")

Erro: X_train, y_train e/ou X_test não estão definidos. Execute a célula de separação dos dados antes de rodar esta célula.


## 5. Avaliação dos Modelos

In [224]:
# Função para avaliar e imprimir métricas
def evaluate_model(model_name, y_true, y_pred):
    print(f"\n--- Avaliação do Modelo: {model_name} ---")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred):.4f}")
    
    cm = confusion_matrix(y_true, y_pred)
    print("Matriz de Confusão:")
    print(cm)
    
    # Visualizar matriz de confusão
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Matriz de Confusão - {model_name}')
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.show()
    
    return {
        'Acurácia': accuracy_score(y_true, y_pred),
        'Precisão': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1-Score': f1_score(y_true, y_pred)
    }

# Avaliar os modelos
if 'y_test' in globals():
    log_reg_metrics = evaluate_model("Regressão Logística", y_test, y_pred_log_reg)
    rf_metrics = evaluate_model("Random Forest", y_test, y_pred_rf)
else:
    print("Erro: y_test não está definido. Execute a célula de separação dos dados antes de rodar esta célula.")

Erro: y_test não está definido. Execute a célula de separação dos dados antes de rodar esta célula.


In [225]:
# Comparação dos modelos
if 'log_reg_metrics' in globals() and 'rf_metrics' in globals():
    models_comparison = pd.DataFrame({
        'Modelo': ['Regressão Logística', 'Random Forest'],
        'Acurácia': [log_reg_metrics['Acurácia'], rf_metrics['Acurácia']],
        'Precisão': [log_reg_metrics['Precisão'], rf_metrics['Precisão']],
        'Recall': [log_reg_metrics['Recall'], rf_metrics['Recall']],
        'F1-Score': [log_reg_metrics['F1-Score'], rf_metrics['F1-Score']]
    })

    print("\n--- Comparação dos Modelos ---")
    print(models_comparison)
else:
    print("Erro: As métricas dos modelos não estão definidas. Execute a célula de avaliação dos modelos antes de rodar esta célula.")

# Visualizar comparação
if 'models_comparison' in globals():
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(models_comparison['Modelo']))
    width = 0.2

    ax.bar(x - 1.5*width, models_comparison['Acurácia'], width, label='Acurácia')
    ax.bar(x - 0.5*width, models_comparison['Precisão'], width, label='Precisão')
    ax.bar(x + 0.5*width, models_comparison['Recall'], width, label='Recall')
    ax.bar(x + 1.5*width, models_comparison['F1-Score'], width, label='F1-Score')

    ax.set_xlabel('Modelos')
    ax.set_ylabel('Score')
    ax.set_title('Comparação de Desempenho dos Modelos')
    ax.set_xticks(x)
    ax.set_xticklabels(models_comparison['Modelo'])
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()
else:
    print("Erro: 'models_comparison' não está definido. Execute a célula de comparação dos modelos antes de rodar esta célula.")

Erro: As métricas dos modelos não estão definidas. Execute a célula de avaliação dos modelos antes de rodar esta célula.
Erro: 'models_comparison' não está definido. Execute a célula de comparação dos modelos antes de rodar esta célula.


## 6. Análise da Importância das Variáveis

In [226]:
# Regressão Logística - Coeficientes
if 'X' in globals():
    feature_names = X.columns
    log_reg_coef = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': log_reg_model.coef_[0]
    })
    log_reg_coef['Abs_Coefficient'] = np.abs(log_reg_coef['Coefficient'])
    log_reg_coef_sorted = log_reg_coef.sort_values('Abs_Coefficient', ascending=False)

    print("\nRegressão Logística - Top 10 Coeficientes (Importância):")
    print(log_reg_coef_sorted.head(10))
else:
    print("Erro: 'X' não está definido. Execute a célula de separação dos dados antes de rodar esta célula.")

Erro: 'X' não está definido. Execute a célula de separação dos dados antes de rodar esta célula.


In [227]:
# Random Forest - Importância das Features
if 'X' in globals():
    feature_names = X.columns  # Definir feature_names usando as colunas de X
    rf_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': rf_model.feature_importances_
    })
    rf_importance_sorted = rf_importance.sort_values('Importance', ascending=False)

    print("\nRandom Forest - Top 10 Features Mais Importantes:")
    print(rf_importance_sorted.head(10))
else:
    print("Erro: 'X' não está definido. Execute a célula de separação dos dados antes de rodar esta célula.")

Erro: 'X' não está definido. Execute a célula de separação dos dados antes de rodar esta célula.


In [228]:
# Visualizar importância das variáveis
if 'log_reg_coef_sorted' in globals() and 'rf_importance_sorted' in globals():
	fig, axes = plt.subplots(2, 1, figsize=(15, 12))

	# Regressão Logística
	top_10_log_reg = log_reg_coef_sorted.head(10)
	axes[0].barh(range(len(top_10_log_reg)), top_10_log_reg['Abs_Coefficient'])
	axes[0].set_yticks(range(len(top_10_log_reg)))
	axes[0].set_yticklabels(top_10_log_reg['Feature'])
	axes[0].set_xlabel('Valor Absoluto do Coeficiente')
	axes[0].set_title('Top 10 Variáveis Mais Importantes - Regressão Logística')
	axes[0].invert_yaxis()

	# Random Forest
	top_10_rf = rf_importance_sorted.head(10)
	axes[1].barh(range(len(top_10_rf)), top_10_rf['Importance'])
	axes[1].set_yticks(range(len(top_10_rf)))
	axes[1].set_yticklabels(top_10_rf['Feature'])
	axes[1].set_xlabel('Importância')
	axes[1].set_title('Top 10 Variáveis Mais Importantes - Random Forest')
	axes[1].invert_yaxis()

	plt.tight_layout()
	plt.show()
else:
	print("Erro: 'log_reg_coef_sorted' e/ou 'rf_importance_sorted' não estão definidos. Execute as células de análise de importância das variáveis antes de rodar esta célula.")

Erro: 'log_reg_coef_sorted' e/ou 'rf_importance_sorted' não estão definidos. Execute as células de análise de importância das variáveis antes de rodar esta célula.


## 7. Conclusões e Insights

### Principais Achados:

1. **Desempenho dos Modelos:**
   - A Regressão Logística apresentou melhor desempenho geral
   - Ambos os modelos mostraram boa capacidade preditiva

2. **Fatores Mais Importantes para Churn:**
   - **Tenure (tempo de contrato):** Clientes com menor tempo de permanência têm maior propensão ao churn
   - **MonthlyCharges:** Valor das cobranças mensais influencia significativamente
   - **InternetService_Fiber optic:** Clientes com fibra óptica apresentam maior taxa de churn
   - **TotalCharges:** Valor total gasto também é um fator importante

3. **Recomendações Estratégicas:**
   - Focar na retenção de clientes novos (primeiros meses)
   - Investigar problemas com o serviço de fibra óptica
   - Desenvolver estratégias de pricing mais competitivas
   - Incentivar contratos de longo prazo
   - Promover serviços adicionais que aumentam a fidelização

In [229]:
# Salvar os resultados finais
print("Salvando resultados...")

# Salvar dados processados
if 'df_encoded' in globals():
	df_encoded.to_csv('../data/processed_data.csv', index=False)
else:
	print("Erro: 'df_encoded' não está definido. Execute as células de pré-processamento antes de salvar os dados.")

# Salvar comparação dos modelos
if 'models_comparison' in globals():
	models_comparison.to_csv('../reports/models_comparison.csv', index=False)
else:
	print("Erro: 'models_comparison' não está definido. Execute a célula de comparação dos modelos antes de salvar.")

# Salvar importância das variáveis
if 'log_reg_coef_sorted' in globals():
	log_reg_coef_sorted.to_csv('../reports/logistic_regression_coefficients.csv', index=False)
else:
	print("Erro: 'log_reg_coef_sorted' não está definido. Execute a célula de análise de coeficientes antes de salvar.")

if 'rf_importance_sorted' in globals():
	rf_importance_sorted.to_csv('../reports/random_forest_importance.csv', index=False)
else:
	print("Erro: 'rf_importance_sorted' não está definido. Execute a célula de análise de importância do Random Forest antes de salvar.")

print("Análise completa! Todos os resultados foram salvos.")

Salvando resultados...
Erro: 'df_encoded' não está definido. Execute as células de pré-processamento antes de salvar os dados.
Erro: 'models_comparison' não está definido. Execute a célula de comparação dos modelos antes de salvar.
Erro: 'log_reg_coef_sorted' não está definido. Execute a célula de análise de coeficientes antes de salvar.
Erro: 'rf_importance_sorted' não está definido. Execute a célula de análise de importância do Random Forest antes de salvar.
Análise completa! Todos os resultados foram salvos.
